# Part II: Model-based Regularization

Now that we know the main caveats of solving inverse problems, it's time to find out how to tackle them! The keyword here will be **regularization**. A regularization method is usually adapted to the noise level $\delta$ with a regularization parameter $\lambda > 0$ and has two essential properties:
* The regularization method is **continuous** (i.e., stable) for all choices of $\lambda$.
* As the noise level tends to zero, the regularization method **converges pointwise to a generalized inverse** (e.g., the operator that maps a noisy measurement to the minimum norm solution).

A popular class of regularization methods can be formulated as a variational problem, where the reconstruction $x^*$ of a noisy measurement $y^{\delta}$ is obtained as the solution of
$$ x^* = \operatorname*{arg\ min}_x \frac{1}{2}\|Ax-y^{\delta}\|^2_2 + \lambda J(x), $$
where the functional $J$ is called the regularization functional and penalizes unwanted behaviour of $x$. In this part of the tutorial, we will see what typical examples of $J$ look like and get an idea of how the minimizer of the above problem can be found. For a stochastical interpretation of the variational formulation stay tuned for the last part of this tutorial.

In [1]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
import operators
import optimizers
import numpy as np
import pywt
import utils

## Tikhonov Regularization

The first regularization functional we consider is $J(x) = \frac{1}{2}\|x\|^2_2 $. An easy interpretation of this would be that **penalizing reconstructions with a large norm prevents the error from exploding** (compare Task 1.4). The resulting variational problem then reads
$$ x^* = \operatorname*{arg\ min}_x \frac{1}{2}\|Ax-y^{\delta}\|^2_2 + \frac{\lambda}{2} \|x\|^2_2.$$

Analytically, the solution of the above problem can be computed as 
$$ x^* = (A^*A + \lambda \operatorname{Id})^{-1} A^*y^\delta,$$
where $A^*$ denotes the adjoint of $A$ and $\operatorname{Id}$ denotes the identity operator. In this form, the method is also known as Tikhonov regularization.

Since we cannot easily access $(A^*A + \lambda \operatorname{Id})^{-1}$, we can perform a simple gradient descent with stepsize $t > 0$ of the form
$$ x \leftarrow x - t \cdot (A^*Ax + \lambda x - A^*y^\delta )$$

### &#128221; <span style="color:darkorange"> Task 2.1 </span>
#### Complete the following algorithm that performs gradient descent to solve the Tikhonov regularization problem.

In [4]:

class gradient_descent(optimizers.optimizer):
    def __init__(self, A, x, y, t=0.1, lamda=1.0, **kwargs):
        super().__init__(**kwargs)
        self.A = A
        self.x = x
        self.y = y
        self.t = t
        self.lamda = lamda
        def energy_fun(x):
            return 0.5* np.linalg.norm(A(x)-y)**2 + lamda*0.5*np.linalg.norm(x, ord=1)
        self.energy_fun=energy_fun

    def step(self,):
           
        ###### TODO's start here #####

        grad = self.A.adjoint(self.A(self.x)) + self.lamda*self.x - self.A.adjoint(self.y) # None # TODO grad should be the gradient A*Ax + \lambda x - A*y^\delta.
                                                                                                  # You can assume that A* the adjoint of self.A is implemented as self.A.adjoint
        self.x = self.x - self.t*grad # None # TODO update self.x by performing a gradient descent with stepsize 'self.t'

        ###### TODO's start here #####

In [5]:
dim = 128
pre_compute_tikh = {}
phantom = utils.get_phantom(dim)

def plot_tikh_reco(lamda, noise_lvl, max_angle):
    num_theta = int(np.ceil(max_angle/180*dim))
    theta = np.linspace(0,max_angle, endpoint= False, num=num_theta)
    R = operators.Radon(theta=theta)
    if (lamda, noise_lvl, max_angle) in pre_compute_tikh:
        x = pre_compute_tikh[(lamda, noise_lvl, max_angle)]
    else:
        sinogram = R(phantom) + np.random.normal(loc = 0, scale= noise_lvl, size = [dim,num_theta])
        
        gd = gradient_descent(R, R.inv(sinogram), sinogram, t=1/(noise_lvl*lamda*10000) if (noise_lvl*lamda > 0 ) else 0.0001, lamda=lamda, verbosity = 0)
        gd.solve()
        
        
        x = gd.x
        pre_compute_tikh[(lamda, noise_lvl, max_angle)] = x
    
    plt.figure()
    plt.imshow(x, vmin=0, vmax = 1)

l_slider = widgets.FloatSlider(min = 0, max = 10., step = 1, value = 0, continuous_update = False)
s_slider = widgets.FloatSlider(min = 0.001, max = .01, step = 0.001, value = 0.001, continuous_update = False)
t_slider = widgets.IntSlider(min = 1, max = 180, step = 10, value = 180, continuous_update = False)
interactive_plot = interactive(plot_tikh_reco, lamda = l_slider, noise_lvl = s_slider, max_angle= t_slider)
display(interactive_plot)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='lamda', max=10.0, step=1.0)…

## Sparsity Promoting Regularization
Another demand on the regularization method could be that the obtained **reconstructions should be simple in a certain sense**. This means that they should be composed of only a few building blocks. 

The two ingredients we will need to enforce this are
* an operator $D$ which decomposes $x$ into building blocks, for example a wavelet decomposition
* the $\|\cdot\|_1$-norm which promotes sparsity, i.e., penalizes non-zero entries of $Dx$.

The resulting variational problem then reads
$$ x^* = \operatorname*{arg\ min}_x \frac{1}{2}\|Ax-y^{\delta}\|^2_2 + \lambda \|Dx\|_1.$$

Since the regularization functional is in general not differentiable, we will use the so-called proximal gradient descent algorithm. The proximal map of a functional $J$ with parameter $\lambda$ is defined as
$$ \operatorname{prox}_{\lambda J}(x) = \operatorname*{arg\ min}_z \frac{1}{2}\|x-z\|_2^2 + \lambda J(z).$$

The proximal gradient descent update then reads 
$$ x \leftarrow \operatorname{prox}_{t\lambda J} (x - 2t \cdot (A^*Ax - A^*y^\delta )).$$


Fortunately, the proximal map of the functionals we consider now is easy to compute with the help of the soft-shrinkage function, which is in fact the proximal map of the $\|\cdot\|_1$-norm. For different choices of $\lambda$ it looks as follows:

In [8]:
v = np.linspace(-10,10, num=100)
def plot_soft(lamda):
    w = optimizers.soft_shrinkage(v, lamda)
    plt.plot(v,w)
    plt.ylim([-10,10])
    plt.title('$prox_{\lambda \|\cdot\|_1}$, $\lambda = $' +str(lamda))
    
    
slider = widgets.FloatSlider(min = 0, max = 10, step = 0.5, value = 1, continuous_update = True)
interactive_plot = interactive(plot_soft, lamda = slider)
display(interactive_plot)

interactive(children=(FloatSlider(value=1.0, description='lamda', max=10.0, step=0.5), Output()), _dom_classes…

### &#128221; <span style="color:darkorange"> Task 2.2 </span>
#### How does the soft-shrinkage function enforce sparsity?

### &#128221; <span style="color:darkorange"> Task 2.3 </span>
#### Try out sparsity promoting regularization for different choices of $D$ (Identity, Haar-Wavelet Transform and Daubechies4-Wavelet Transform) and describe how the reconstructions change depending on $D$.
To interpret the approaches using wavelet decompositions it might be useful to look for an image of the corresponding wavelets online. 

In [5]:
dim = 128
pre_compute_sparse = {}
phantom = utils.get_phantom(dim)

def plot_sparse_reco(D_idx, lamda, noise_lvl, max_angle):
    num_theta = int(np.ceil(max_angle/180*dim))
    theta = np.linspace(0,max_angle, endpoint= False, num=num_theta)
    R = operators.Radon(theta=theta)
    if (D_idx, lamda, noise_lvl, max_angle) in pre_compute_sparse:
        x = pre_compute_sparse[(D_idx, lamda, noise_lvl, max_angle)]
    else:
        sinogram = R(phantom) + np.random.normal(loc = 0, scale= noise_lvl, size = [dim,num_theta])
        opti = None
        if D_idx == 0:
            opti = optimizers.ista_L1(R, R.inv(sinogram), sinogram, t=1/(noise_lvl*lamda*10000) if (noise_lvl*lamda > 0 ) else 0.0001, lamda=lamda,  verbosity = 0)
        elif D_idx == 1:
            opti = optimizers.ista_wavelets(R, R.inv(sinogram), sinogram, wave = pywt.Wavelet('haar'), t=1/(noise_lvl*lamda*10000) if (noise_lvl*lamda > 0 ) else 0.0001, lamda=lamda,  verbosity = 0)
        elif D_idx == 2:
            opti = optimizers.ista_wavelets(R, R.inv(sinogram), sinogram, wave = pywt.Wavelet('db4'), t=1/(noise_lvl*lamda*10000) if (noise_lvl*lamda > 0 ) else 0.0001, lamda=lamda,  verbosity = 0)

        opti.solve()
        
        x = opti.x

        pre_compute_sparse[(D_idx, lamda, noise_lvl, max_angle)] = x
    
    plt.figure()
    plt.imshow(x, vmin=0, vmax = 1)

idx_toggle = widgets.ToggleButtons(
    options=[('Identity',0), ('Haar-Wavelet',1), ('Daubechies4-Wavelet',2)],
    description='Decomposing operator D',
    disabled=False
)
l_slider = widgets.FloatSlider(min = 0, max = 0.5, step = 0.01, value = 0, continuous_update = False)
s_slider = widgets.FloatSlider(min = 0.001, max = .01, step = 0.001, value = 0.001, continuous_update = False)
t_slider = widgets.IntSlider(min = 1, max = 180, step = 10, value = 180, continuous_update = False)
interactive_plot = interactive(plot_sparse_reco, D_idx = idx_toggle, lamda = l_slider, noise_lvl = s_slider, max_angle= t_slider)

display(interactive_plot)

interactive(children=(ToggleButtons(description='Decomposing operator D', options=(('Identity', 0), ('Haar-Wav…

## Total Variation Regularization

A regularization functional that works particularly well for image processing is the Total Variation (TV). It can be written as
$$\operatorname{TV}(x) = \| \nabla x\|_1$$ 
and is thus also a sparsity promoting functional. However, we cannot compute its proximal map as easy as above and thus have to use a more complicated optimization algorithm.

### &#128221; <span style="color:darkorange"> Task 2.4 </span>
#### Try to find out in which sense TV regularization promotes sparsity. The following slider showing some TV reconstructions might help you.

In [6]:
dim = 128
pre_compute_tv = {}
phantom = utils.get_phantom(dim)

def plot_tv_reco(lamda, noise_lvl, max_angle):
    num_theta = int(np.ceil(max_angle/180*dim))
    theta = np.linspace(0,max_angle, endpoint= False, num=num_theta)
    R = operators.Radon(theta=theta)
    if (lamda, noise_lvl, max_angle) in pre_compute_sparse:
        x = pre_compute_sparse[(lamda, noise_lvl, max_angle)]
    else:
        sinogram = R(phantom) + np.random.normal(loc = 0, scale= noise_lvl, size = [dim,num_theta])
        def energy_fun(x):
            return np.linalg.norm(R(x) - sinogram)**2 + lamda * operators.TV()(x)
        
        sBTV = optimizers.split_Bregman_TV(R, sinogram, R.inv(sinogram), gamma=1.0, 
                        energy_fun=energy_fun, lamda = lamda,
                        max_it = 10,
                        max_inner_it = 2, verbosity = 0)
        sBTV.solve()
        
        x = sBTV.x

        pre_compute_tv[(lamda, noise_lvl, max_angle)] = x
    
    plt.figure()
    plt.imshow(x, vmin=0, vmax = 1)

l_slider = widgets.FloatSlider(min = 0, max = 0.1, step = 0.001, value = 0, continuous_update = False)
s_slider = widgets.FloatSlider(min = 0.001, max = .01, step = 0.001, value = 0.001, continuous_update = False)
t_slider = widgets.IntSlider(min = 1, max = 180, step = 10, value = 180, continuous_update = False)
interactive_plot = interactive(plot_tv_reco, lamda = l_slider, noise_lvl = s_slider, max_angle= t_slider)

display(interactive_plot)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='lamda', max=0.1, step=0.001…